In [1]:
from elasticsearch import Elasticsearch

In [2]:
es  = Elasticsearch(
    'https://localhost:9200',
    basic_auth=('elastic' , 'epd1di6ZdaxYfLp0qmep'),
    ca_certs='C:\elasticsearch\elasticsearch-8.13.4\config\certs\http_ca.crt'
)
es.ping()

True

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('myntra_products_catalog.csv').loc[:499]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna('None', inplace=True)

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\91741\Documents\search_bar\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\91741\Documents\search_bar\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\91741\Documents\search_bar\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
df['description_vector'] = df['Description'].apply(lambda x: model.encode(x))

In [9]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,description_vector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645884, -0.0026341917, -0.0035884108, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.0246607, -0.028755352, -0.020332512, 0.034..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943244, 0.081827946, 0.048335165, -0.00..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098729, -0.010285393, 0.009487305, -0.0..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746598, 0.0062096263, 0.021813963, 0.02..."


In [10]:
es.ping()

True

In [11]:
from indexMapping import iMapping

In [12]:
es.indices.create(index='all_product' , mappings=iMapping)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [all_product/kx8IeWAZT0qbzJjHFt-4ZA] already exists')

In [ ]:
record_list = df.to_dict('records')

In [ ]:
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'description_vector': array([ 2.76458841e-02, -2.63419165e-03, -3.58841079e-03,  5.13588041e-02,
         3.09661552e-02,  1.40507389e-02,  7.27059739e-03,  3.13872173e-02,
        -6.23786747e-02, -3.82885640e-03,  3.15214284e-02,  7.55471513e-02,
         2.12641782e-03,  4.64892872e-02,  5.07448874e-02, -1.71942208e-02,
         1.22892950e-02, -1.95682496e-02, -9.

In [ ]:
for record in record_list:

    try:
        es.index(index='all_product' ,  document=record , id=record['ProductID'])

    except Exception as e:
        print(e)

In [ ]:
es.count(index='all_product')

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
input_keyword = ' Shoes'
vector_of_input_keyword = model.encode(input_keyword)  # Assuming model is a SentenceTransformer model

query = {
    'field' : 'description_vector',
    'query_vector' : vector_of_input_keyword,
    'k' : 2 ,
    'num_candidates' : 500,
    
}

res = es.knn_search(index='all_product', knn=query, source=['ProductName', 'Description'])
res['hits']['hits']


C:\Users\91741\AppData\Local\Temp\ipykernel_10036\1297393284.py:12: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index='all_product', knn=query, source=['ProductName', 'Description'])


[{'_index': 'all_product',
  '_id': '10006031',
  '_score': 0.55765337,
  '_source': {'ProductName': 'ID Men Brown Solid Leather Mid-Top Sneakers',
   'Description': 'A pair of round-toe brown sneakers, has mid-top styling, lace-up detailLeather upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_product',
  '_id': '10001425',
  '_score': 0.55642545,
  '_source': {'ProductName': 'Shoe Couture Women White & Black Applique Detail Sneakers',
   'Description': 'A pair of round-toe white & black sneakers, has regular styling, lace-up detailSynthetic upper with applique detailCushioned footbedTextured and patterned outsoleWarranty: 1 monthWarranty provided by brand/manufacturer'}}]